In [1]:
import pandas as pd
import os
from tqdm.auto import tqdm
from PIL import Image

In [3]:
#path='data/baselinetest_conf/labels/'
path='data/fold1_pred/'
img_path='data/testframe/'
out='fold1.txt'

In [4]:
def rm_overlap(df,iou_threshold=0.75):
    for index1, row1 in df.iterrows():
#         count=0
        bb1,bb2={},{}
        bb1['x1'],bb1['y1']=row1['bb_left'],row1['bb_top']
        bb1['x2'],bb1['y2']=row1['bb_left']+row1['width'],row1['bb_top']+row1['height']
        for index2, row2 in df.iterrows():
            if index2==index1 or row1['class']!= row2['class']: continue
            bb2['x1'],bb2['y1']=row2['bb_left'],row2['bb_top']
            bb2['x2'],bb2['y2']=row2['bb_left']+row2['width'],row2['bb_top']+row2['height']
            if get_iou(bb1,bb2)>iou_threshold:
                #count+=1
                df=df.drop(index2)
#         if count:
#             print(f'overlap {count}')
    return df
            
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [5]:
def rm_mins(df,mins=1681):
    for index,row in df.iterrows():
        s=row['width']*row['height']
        if s<mins:
            df=df.drop(index)
    return df
    

In [6]:
appended_data = []
conf_thres=0.5
for i in tqdm(sorted(os.listdir(path))):
    try:
        df=pd.read_csv(path+i,delimiter=' ',header=None)
    except:
        print(i)
        break
    df.columns=['class', 'x_centre', 'y_centre', 'width', 'height','confidence']
    df['vid']=int(i[:3])
    df['frame_id']=int(i[4:-4])
    df['bb_left']=df['x_centre']-df['width']/2.0
    df['bb_top']=df['y_centre']-df['height']/2.0
    df['class']=df['class']+1
    df = df[['vid','frame_id','bb_left','bb_top', 'width', 'height','class','confidence']]
    #post processing
#     df = df.drop(df[df['confidence'] <= 0.5].index)
    df=rm_overlap(df)
    #format
#     img='%03d'%int(i[:3])+'_'+'%03d'%int(i[4:-4])+'.jpg'
#     im = Image.open(img_path+img)
    width, height = (1920,1080)
    df['width']=(df['width']*width).astype(int)
    df['bb_left']=(df['bb_left']*width).astype(int)
    df['height']=(df['height']*height).astype(int)
    df['bb_top']=(df['bb_top']*height).astype(int)
    #df=rm_mins(df)
    #checksize
#     for index,row in df.iterrows():
#         df.at[index, 'width'] = min(df.at[index,'width'],width-df.at[index,'bb_left'])
#         df.at[index, 'height'] = min(df.at[index,'height'],height-df.at[index,'bb_top'])
    appended_data.append(df)
df    

  0%|          | 0/15412 [00:00<?, ?it/s]

,vid,frame_id,bb_left,bb_top,width,height,class,confidence
0,100,99,0,853,81,207,1,0.558793
1,100,99,1009,945,160,135,1,0.734504
2,100,99,631,470,66,91,1,0.786988
3,100,99,626,373,81,143,2,0.802847
4,100,99,997,802,163,271,2,0.819049


In [7]:
appended_data = pd.concat(appended_data)
appended_data=appended_data.sort_values(['vid', 'frame_id'], ascending=[True, True])
appended_data

,vid,frame_id,bb_left,bb_top,width,height,class,confidence
0,1,1,356,371,49,100,2,0.612928
1,1,1,523,342,49,101,2,0.674376
2,1,1,521,388,54,84,1,0.729236
0,1,2,529,348,51,103,2,0.514381
1,1,2,361,418,51,84,1,0.691925
...,...,...,...,...,...,...,...,...
1,100,200,956,316,52,73,1,0.721686
2,100,200,889,400,85,157,3,0.785852
3,100,200,952,246,60,111,2,0.811111
4,100,200,909,325,67,115,2,0.844185


In [8]:
appended_data.to_csv(out,index=False,header=None)

# Oversize

In [9]:
new_df=pd.read_csv(out)
new_df.columns=['vid','frame_id','bb_left','bb_top', 'width', 'height','class','confidence']
    

In [10]:
maxi=0
maxj=0
for index,row in new_df.iterrows():
    maxi=max(maxi,row['bb_left']+row['width'])
    maxj=max(maxj,row['bb_top']+row['height'])
    if row['bb_left']+row['width']>1920:
        print(row)
    if row['bb_top']+row['height']>1080:
        print(row)
print(maxi,maxj)

1920.0 1080.0


# Smallest box 

In [11]:
# gt=pd.read_csv(out)
# gt.columns=['vid','frame_id','tracking','bb_left','bb_top', 'width', 'height','class']
mins=100000
for index,row in new_df.iterrows():
    s=row['width']*row['height']
    mins=min(mins,s)
print(mins)

525.0
